In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Resizing, Rescaling, Dense
from tensorflow.keras import Sequential
from PIL import Image

In [29]:

cancer_types = ['pituitary','notumor','glioma','meningioma']
dataset_types = ['Training','Testing']

# Dataset sizes
print('Total images available...')
for dataset in dataset_types:
    for cancer in cancer_types:
        print(f'{cancer.capitalize()} {dataset.lower()}: {len(os.listdir(os.path.join('data',dataset,cancer)))}')



Total images available...
Pituitary training: 1457
Notumor training: 1595
Glioma training: 1321
Meningioma training: 1339
Pituitary testing: 300
Notumor testing: 405
Glioma testing: 300
Meningioma testing: 306


In [42]:
image = tf.io.read_file('data/Training/notumor/Tr-no_0021.jpg')
image_tensor = tf.image.decode_jpeg(image, channels = 1)
print(len(image_tensor))
print(len(image_tensor[0]))

# Resize 
preprocess_image = Sequential([
    Resizing(150,150),
    Rescaling(1.0/255.0)
])




830
1024
